## Collecting our All our Data using AWS and API Requests

## Import Libraries

In [1]:
import requests
import os
import io
import boto3
import time
import csv
import sys
from concurrent.futures import ThreadPoolExecutor, as_completed
from dotenv import load_dotenv
load_dotenv()

True

## Starting our AWS Session

In [2]:
session = boto3.Session(profile_name='personal-onemap')
s3 = session.resource('s3')

## Import API Authorisation Key

In [3]:
# For OneMap Account Key
account_key = os.getenv('ONEMAP_KEY')
headers = {'Authorization': f'Bearer {account_key}'}


## Count number of areas in Singapore

In [4]:
api_url = "https://www.onemap.gov.sg/api/public/popapi/getPlanningareaNames?year=2019" # Latest area info from 2019

response = requests.get(api_url, headers=headers, timeout=1)
data = response.json()

# Count number of area codes
area_count = sum(1 for item in data if "pln_area_n" in item)
print(f'Number of areas: {area_count}')

# Save our area codes into List
area_list = [item['pln_area_n'] for item in data if "pln_area_n" in item]
print(area_list)


Number of areas: 55
['BEDOK', 'BUKIT TIMAH', 'BUKIT BATOK', 'BUKIT MERAH', 'CENTRAL WATER CATCHMENT', 'DOWNTOWN CORE', 'CHANGI', 'CHANGI BAY', 'LIM CHU KANG', 'BOON LAY', 'WESTERN WATER CATCHMENT', 'WOODLANDS', 'MARINE PARADE', 'NEWTON', 'NORTH-EASTERN ISLANDS', 'ORCHARD', 'PASIR RIS', 'PIONEER', 'PUNGGOL', 'QUEENSTOWN', 'SEMBAWANG', 'SIMPANG', 'TAMPINES', 'TANGLIN', 'TUAS', 'WESTERN ISLANDS', 'SOUTHERN ISLANDS', 'BUKIT PANJANG', 'BISHAN', 'ANG MO KIO', 'GEYLANG', 'STRAITS VIEW', 'JURONG EAST', 'HOUGANG', 'JURONG WEST', 'CHOA CHU KANG', 'KALLANG', 'MANDAI', 'TENGAH', 'MARINA EAST', 'MARINA SOUTH', 'MUSEUM', 'NOVENA', 'OUTRAM', 'PAYA LEBAR', 'RIVER VALLEY', 'ROCHOR', 'SELETAR', 'SENGKANG', 'SERANGOON', 'CLEMENTI', 'TOA PAYOH', 'SINGAPORE RIVER', 'SUNGEI KADUT', 'YISHUN']


## Test API Output for Economic Status

In [5]:
response = requests.get(f"https://www.onemap.gov.sg/api/public/popapi/getEconomicStatus?planningArea=Bedok&year=2010", headers=headers, timeout=1)
data = response.json()


print(response)
print(data)

<Response [200]>
[{'planning_area': 'Bedok', 'employed': 83567, 'unemployed': 3748, 'inactive': 33195, 'year': 2010, 'gender': 'Male'}, {'planning_area': 'Bedok', 'employed': 64100, 'unemployed': 2802, 'inactive': 59608, 'year': 2010, 'gender': 'Female'}]


## Slow Method: Synchronous Requests (Economic Status)

In [6]:
years = ['2000', '2010', '2015', '2020'] 

# Iterate over each year and area
for year in years:
        csv_file = io.StringIO()
        headers_fetched = False
        for area in area_list:
            response = requests.get(f"https://www.onemap.gov.sg/api/public/popapi/getEconomicStatus?planningArea={area}&year={year}", headers=headers, timeout=1.5)
            data = response.json()

            if response.status_code == 200 and data:
                # Initialize CSV writer with headers from the first successful response
                if not headers_fetched:
                    if 'Result' in data and data['Result'] == "No Data Available!":
                        continue  # Skip to the next area if no data available
                    else:
                        fieldnames = data[0].keys()  
                        csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                        csv_writer.writeheader()
                        headers_fetched = True
                
                if 'Result' not in data or data['Result'] != "No Data Available!":
                    for item in data:
                        csv_writer.writerow(item)
                    print(f"Completed: {area} in {year} has been logged.")
                else:
                    print(f"Skipped: {area} in {year} has no results.")
            else:
                print(f"Failed to process {area} in {year}.")

        if headers_fetched:  # Only proceed if we successfully fetched headers and wrote some data
            print(f"All areas processed for {year}.")

            csv_file.seek(0)


            s3_file_path = f'Economic_Status_Data/' + f'data_year_{year}.csv'

            # Upload to S3
            s3.Object("onemap-s3", s3_file_path).put(Body=csv_file.getvalue(), ContentType='text/csv')
            print(f"Successfully uploaded to S3: {s3_file_path}")
        
        csv_file.close()


Completed: BEDOK in 2000 has been logged.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Skipped: CENTRAL WATER CATCHMENT in 2000 has no results.
Completed: DOWNTOWN CORE in 2000 has been logged.
Completed: CHANGI in 2000 has been logged.
Skipped: CHANGI BAY in 2000 has no results.
Skipped: LIM CHU KANG in 2000 has no results.
Skipped: BOON LAY in 2000 has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 has no results.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Completed: NEWTON in 2000 has been logged.
Skipped: NORTH-EASTERN ISLANDS in 2000 has no results.
Skipped: ORCHARD in 2000 has no results.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Skipped: PUNGGOL in 2000 has no results.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been logged.
Skipped: SIMPANG in

## Fast Method: Asynchronous Multi-Threaded Requests (Economic Status)

In [7]:
years = ['2000', '2010', '2015', '2020'] 
max_requests_per_minute = 200
request_interval = 60.0 / max_requests_per_minute

exception_occurred = False

def fetch_data(year, area, start_time, index):
    global exception_occurred # To prevent missing values
    if exception_occurred:  
        return None
    time.sleep(max(0, start_time + index * request_interval - time.time()))  # Rate limiting
    try:
        response = requests.get(f"https://www.onemap.gov.sg/api/public/popapi/getEconomicStatus?planningArea={area}&year={year}", headers=headers, timeout=1.5)
        data = response.json()
        if response.status_code == 200 and data:
            if 'Result' in data and data['Result'] == "No Data Available!":
                print(f"Skipped: {area} in {year} has no results.")
                return None
            else:
                print(f"Completed: {area} in {year} has been logged.")
                return data
        else:
            print(f"Failed to process {area} in {year}.")
            return None
    except Exception as e:
        print(f"Exception for {area} in {year}: {e}")
        exception_occurred = True 
        return None

def process_year(year):
    global exception_occurred # To prevent missing values
    csv_file = io.StringIO()
    fieldnames = []  
    csv_writer = None

    start_time = time.time()
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(fetch_data, year, area, start_time, index): area for index, area in enumerate(area_list)}
        for future in as_completed(futures):
            if exception_occurred:  # Stop processing if an exception occurred
                print("Halting due to exception.")
                executor.shutdown(wait=False)  
                csv_file.close()
                sys.exit(1)  # Exit the script

            data = future.result()
            if data:
                if not fieldnames:
                    fieldnames = data[0].keys()
                    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                    csv_writer.writeheader()
                for item in data:
                    csv_writer.writerow(item)

    if csv_writer: 
        csv_file.seek(0)
        s3_file_path = f'Economic_Status_Data_Multi/' + f'data_year_{year}.csv'
        print(f"Successfully prepared data for S3 upload: {s3_file_path}")
        s3.Object("onemap-s3", s3_file_path).put(Body=csv_file.getvalue(), ContentType='text/csv')
        csv_file.close()

for year in years:
    if exception_occurred:  # Check for missing value occurrance
        print("Script stopped due to an earlier exception.")
        break
    process_year(year)

Completed: BEDOK in 2000 has been logged.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Skipped: CENTRAL WATER CATCHMENT in 2000 has no results.
Completed: DOWNTOWN CORE in 2000 has been logged.
Completed: CHANGI in 2000 has been logged.
Skipped: LIM CHU KANG in 2000 has no results.
Skipped: CHANGI BAY in 2000 has no results.
Skipped: BOON LAY in 2000 has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 has no results.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Completed: NEWTON in 2000 has been logged.
Skipped: NORTH-EASTERN ISLANDS in 2000 has no results.
Skipped: ORCHARD in 2000 has no results.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Skipped: PUNGGOL in 2000 has no results.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been logged.
Skipped: SIMPANG in

# Global Functions

In [23]:
exception_occurred = False
def fetch_data(api_config, year, area, start_time, index):
    global exception_occurred
    if exception_occurred:
        return None
    time.sleep(max(0, start_time + index * request_interval - time.time()))  # Rate limiting
    try:
        api_url = f"{api_config['url']}?planningArea={area}&year={year}"
        response = requests.get(api_url, headers=headers, timeout=1.5)
        data = response.json()
        if response.status_code == 200 and data:
            if 'Result' in data and data['Result'] == "No Data Available!":
                print(f"Skipped: {area} in {year} has no results.")
                return None
            else:
                print(f"Completed: {area} in {year} has been logged.")
                return data
        else:
            print(f"Failed to process {area} in {year}.")
            return None
    except Exception as e:
        print(f"Exception for {area} in {year}: {e}")
        exception_occurred = True
        return None
    
def process_year(api_config, year):
    global exception_occurred
    csv_file = io.StringIO()
    fieldnames = []
    csv_writer = None

    start_time = time.time()
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(fetch_data, api_config, year, area, start_time, index): area for index, area in enumerate(area_list)}
        for future in as_completed(futures):
            if exception_occurred:
                print("Halting due to exception.")
                executor.shutdown(wait=False)
                csv_file.close()
                sys.exit(1)  # Exit the script

            data = future.result()
            if data:
                if not fieldnames:
                    fieldnames = data[0].keys()
                    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                    csv_writer.writeheader()
                for item in data:
                    csv_writer.writerow(item)

    if csv_writer:
        csv_file.seek(0)
        s3_file_path = f'{api_config["name"]}/' + f'data_year_{year}.csv'
        print(f"Successfully prepared data for S3 upload: {s3_file_path}")
        s3.Object("onemap-s3", s3_file_path).put(Body=csv_file.getvalue(), ContentType='text/csv')
        csv_file.close()



## Asynchronous Multi-Threaded Requests (Education Status)

In [9]:
year = ['2000', '2010', '2015', '2020'] 
api_configs = [
    {
        "name": "Education_Status_Data",
        "url": "https://www.onemap.gov.sg/api/public/popapi/getEducationAttending"
    }
]
for api_config in api_configs:
    for year in years:
        if exception_occurred:
            print("Script stopped due to an earlier exception.")
            break
        process_year(api_config, year)
        

Completed: BEDOK in 2000 has been logged.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Skipped: CENTRAL WATER CATCHMENT in 2000 has no results.
Completed: DOWNTOWN CORE in 2000 has been logged.
Completed: CHANGI in 2000 has been logged.
Skipped: CHANGI BAY in 2000 has no results.
Skipped: LIM CHU KANG in 2000 has no results.
Skipped: BOON LAY in 2000 has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 has no results.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Completed: NEWTON in 2000 has been logged.
Skipped: NORTH-EASTERN ISLANDS in 2000 has no results.
Skipped: ORCHARD in 2000 has no results.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Skipped: PUNGGOL in 2000 has no results.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been logged.
Skipped: SIMPANG in

## Asynchronous Multi-Threaded Requests (Ethnic Distribution Data)

In [10]:
year = ['2000', '2010', '2015', '2020'] 
api_configs = [
    {
        "name": "Ethnic Distribution_Data",
        "url": "https://www.onemap.gov.sg/api/public/popapi/getEthnicGroup"
    }
]
for api_config in api_configs:
    for year in years:
        if exception_occurred:
            print("Script stopped due to an earlier exception.")
            break
        process_year(api_config, year)

Completed: BEDOK in 2000 has been logged.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Skipped: CENTRAL WATER CATCHMENT in 2000 has no results.
Skipped: DOWNTOWN CORE in 2000 has no results.
Skipped: CHANGI in 2000 has no results.
Skipped: CHANGI BAY in 2000 has no results.
Skipped: LIM CHU KANG in 2000 has no results.
Skipped: BOON LAY in 2000 has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 has no results.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Skipped: NEWTON in 2000 has no results.
Skipped: NORTH-EASTERN ISLANDS in 2000 has no results.
Skipped: ORCHARD in 2000 has no results.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Skipped: PUNGGOL in 2000 has no results.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been logged.
Skipped: SIMPANG in 2000 has

## Asynchronous Multi-Threaded Requests (Work Income for Household monthly)

In [11]:
year = ['2000', '2010', '2015', '2020'] 
api_configs = [
    {
        "name": "Work_Income_Household_Monthly_Data",
        "url": "https://www.onemap.gov.sg/api/public/popapi/getHouseholdMonthlyIncomeWork"
    }
]
for api_config in api_configs:
    for year in years:
        if exception_occurred:
            print("Script stopped due to an earlier exception.")
            break
        process_year(api_config, year)

Completed: BEDOK in 2000 has been logged.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Skipped: CENTRAL WATER CATCHMENT in 2000 has no results.
Completed: DOWNTOWN CORE in 2000 has been logged.
Completed: CHANGI in 2000 has been logged.
Skipped: CHANGI BAY in 2000 has no results.
Skipped: LIM CHU KANG in 2000 has no results.
Skipped: BOON LAY in 2000 has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 has no results.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Completed: NEWTON in 2000 has been logged.
Skipped: NORTH-EASTERN ISLANDS in 2000 has no results.
Skipped: ORCHARD in 2000 has no results.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Skipped: PUNGGOL in 2000 has no results.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been logged.
Skipped: SIMPANG in

## Asynchronous Multi-Threaded Requests (Household Size Data)

In [12]:
year = ['2000', '2010', '2015', '2020'] 
api_configs = [
    {
        "name": "Household_Size_Data",
        "url": "https://www.onemap.gov.sg/api/public/popapi/getHouseholdSize"
    }
]
for api_config in api_configs:
    for year in years:
        if exception_occurred:
            print("Script stopped due to an earlier exception.")
            break
        process_year(api_config, year)

Completed: BEDOK in 2000 has been logged.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Skipped: CENTRAL WATER CATCHMENT in 2000 has no results.
Completed: DOWNTOWN CORE in 2000 has been logged.
Completed: CHANGI in 2000 has been logged.
Skipped: CHANGI BAY in 2000 has no results.
Skipped: LIM CHU KANG in 2000 has no results.
Skipped: BOON LAY in 2000 has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 has no results.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Completed: NEWTON in 2000 has been logged.
Skipped: NORTH-EASTERN ISLANDS in 2000 has no results.
Skipped: ORCHARD in 2000 has no results.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Skipped: PUNGGOL in 2000 has no results.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been logged.
Skipped: SIMPANG in

## Asynchronous Multi-Threaded Requests (Household Structure Data)

In [13]:
year = ['2000', '2010', '2015', '2020'] 
api_configs = [
    {
        "name": "Household_Structure_Data",
        "url": "https://www.onemap.gov.sg/api/public/popapi/getHouseholdStructure"
    }
]
for api_config in api_configs:
    for year in years:
        if exception_occurred:
            print("Script stopped due to an earlier exception.")
            break
        process_year(api_config, year)

Completed: BEDOK in 2000 has been logged.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Skipped: CENTRAL WATER CATCHMENT in 2000 has no results.
Completed: DOWNTOWN CORE in 2000 has been logged.
Completed: CHANGI in 2000 has been logged.
Skipped: CHANGI BAY in 2000 has no results.
Skipped: LIM CHU KANG in 2000 has no results.
Skipped: BOON LAY in 2000 has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 has no results.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Completed: NEWTON in 2000 has been logged.
Skipped: NORTH-EASTERN ISLANDS in 2000 has no results.
Skipped: ORCHARD in 2000 has no results.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Skipped: PUNGGOL in 2000 has no results.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been logged.
Skipped: SIMPANG in

## Asynchronous Multi-Threaded Requests (Income from Work Data)

In [14]:
year = ['2000', '2010', '2015', '2020'] 
api_configs = [
    {
        "name": "Income_Work_Data",
        "url": "https://www.onemap.gov.sg/api/public/popapi/getIncomeFromWork"
    }
]
for api_config in api_configs:
    for year in years:
        if exception_occurred:
            print("Script stopped due to an earlier exception.")
            break
        process_year(api_config, year)

Completed: BEDOK in 2000 has been logged.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Skipped: CENTRAL WATER CATCHMENT in 2000 has no results.
Completed: DOWNTOWN CORE in 2000 has been logged.
Completed: CHANGI in 2000 has been logged.
Skipped: CHANGI BAY in 2000 has no results.
Skipped: LIM CHU KANG in 2000 has no results.
Skipped: BOON LAY in 2000 has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 has no results.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Completed: NEWTON in 2000 has been logged.
Skipped: NORTH-EASTERN ISLANDS in 2000 has no results.
Skipped: ORCHARD in 2000 has no results.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Skipped: PUNGGOL in 2000 has no results.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been logged.
Skipped: SIMPANG in

## Asynchronous Multi-Threaded Requests (Industry of Population)

In [15]:
year = ['2000', '2010', '2015', '2020'] 
api_configs = [
    {
        "name": "Industry_Population_Data",
        "url": "https://www.onemap.gov.sg/api/public/popapi/getIndustry"
    }
]
for api_config in api_configs:
    for year in years:
        if exception_occurred:
            print("Script stopped due to an earlier exception.")
            break
        process_year(api_config, year)

Completed: BEDOK in 2000 has been logged.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Skipped: CENTRAL WATER CATCHMENT in 2000 has no results.
Completed: DOWNTOWN CORE in 2000 has been logged.
Completed: CHANGI in 2000 has been logged.
Skipped: CHANGI BAY in 2000 has no results.
Skipped: LIM CHU KANG in 2000 has no results.
Skipped: BOON LAY in 2000 has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 has no results.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Completed: NEWTON in 2000 has been logged.
Skipped: NORTH-EASTERN ISLANDS in 2000 has no results.
Skipped: ORCHARD in 2000 has no results.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Skipped: PUNGGOL in 2000 has no results.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been logged.
Skipped: SIMPANG in

## Asynchronous Multi-Threaded Requests (Language Literacy Data)

In [16]:
year = ['2000', '2010', '2015', '2020'] 
api_configs = [
    {
        "name": "Language_Literacy_Data",
        "url": "https://www.onemap.gov.sg/api/public/popapi/getLanguageLiterate"
    }
]
for api_config in api_configs:
    for year in years:
        if exception_occurred:
            print("Script stopped due to an earlier exception.")
            break
        process_year(api_config, year)

Skipped: BEDOK in 2000 has no results.
Skipped: BUKIT TIMAH in 2000 has no results.
Skipped: BUKIT BATOK in 2000 has no results.
Skipped: BUKIT MERAH in 2000 has no results.
Skipped: CENTRAL WATER CATCHMENT in 2000 has no results.
Skipped: DOWNTOWN CORE in 2000 has no results.
Skipped: CHANGI in 2000 has no results.
Skipped: CHANGI BAY in 2000 has no results.
Skipped: LIM CHU KANG in 2000 has no results.
Skipped: BOON LAY in 2000 has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 has no results.
Skipped: WOODLANDS in 2000 has no results.
Skipped: MARINE PARADE in 2000 has no results.
Skipped: NEWTON in 2000 has no results.
Skipped: NORTH-EASTERN ISLANDS in 2000 has no results.
Skipped: ORCHARD in 2000 has no results.
Skipped: PASIR RIS in 2000 has no results.
Skipped: PIONEER in 2000 has no results.
Skipped: PUNGGOL in 2000 has no results.
Skipped: QUEENSTOWN in 2000 has no results.
Skipped: SEMBAWANG in 2000 has no results.
Skipped: SIMPANG in 2000 has no results.
Skipped: TAMPI

## Asynchronous Multi-Threaded Requests (Marital Status Data)

In [17]:
year = ['2000', '2010', '2015', '2020'] 
api_configs = [
    {
        "name": "Marital_Status_Data",
        "url": "https://www.onemap.gov.sg/api/public/popapi/getMaritalStatus"
    }
]
for api_config in api_configs:
    for year in years:
        if exception_occurred:
            print("Script stopped due to an earlier exception.")
            break
        process_year(api_config, year)

Completed: BEDOK in 2000 has been logged.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Skipped: CENTRAL WATER CATCHMENT in 2000 has no results.
Completed: DOWNTOWN CORE in 2000 has been logged.
Completed: CHANGI in 2000 has been logged.
Skipped: CHANGI BAY in 2000 has no results.
Skipped: LIM CHU KANG in 2000 has no results.
Skipped: BOON LAY in 2000 has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 has no results.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Completed: NEWTON in 2000 has been logged.
Skipped: NORTH-EASTERN ISLANDS in 2000 has no results.
Skipped: ORCHARD in 2000 has no results.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Skipped: PUNGGOL in 2000 has no results.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been logged.
Skipped: SIMPANG in

## Asynchronous Multi-Threaded Requests (Mode of Transport to School)

In [24]:
year = ['2000', '2010', '2015', '2020'] 
api_configs = [
    {
        "name": "Mode_Transport_School_Data",
        "url": "https://www.onemap.gov.sg/api/public/popapi/getModeOfTransportSchool"
    }
]
for api_config in api_configs:
    for year in years:
        if exception_occurred:
            print("Script stopped due to an earlier exception.")
            break
        process_year(api_config, year)

Completed: BEDOK in 2000 has been logged.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Skipped: CENTRAL WATER CATCHMENT in 2000 has no results.
Completed: DOWNTOWN CORE in 2000 has been logged.
Completed: CHANGI in 2000 has been logged.
Skipped: CHANGI BAY in 2000 has no results.
Skipped: LIM CHU KANG in 2000 has no results.
Skipped: BOON LAY in 2000 has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 has no results.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Completed: NEWTON in 2000 has been logged.
Skipped: NORTH-EASTERN ISLANDS in 2000 has no results.
Skipped: ORCHARD in 2000 has no results.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Skipped: PUNGGOL in 2000 has no results.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been logged.
Skipped: SIMPANG in

## Asynchronous Multi-Threaded Requests (Mode of Transport to Work)

In [25]:
year = ['2000', '2010', '2015', '2020'] 
api_configs = [
    {
        "name": "Mode_Transport_Work_Data",
        "url": "https://www.onemap.gov.sg/api/public/popapi/getModeOfTransportWork"
    }
]
for api_config in api_configs:
    for year in years:
        if exception_occurred:
            print("Script stopped due to an earlier exception.")
            break
        process_year(api_config, year)

Completed: BEDOK in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Skipped: CENTRAL WATER CATCHMENT in 2000 has no results.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: DOWNTOWN CORE in 2000 has been logged.
Completed: CHANGI in 2000 has been logged.
Skipped: CHANGI BAY in 2000 has no results.
Skipped: LIM CHU KANG in 2000 has no results.
Skipped: BOON LAY in 2000 has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 has no results.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Completed: NEWTON in 2000 has been logged.
Skipped: NORTH-EASTERN ISLANDS in 2000 has no results.
Skipped: ORCHARD in 2000 has no results.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Skipped: PUNGGOL in 2000 has no results.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been logged.
Skipped: SIMPANG in

## Asynchronous Multi-Threaded Requests (Age Data)

In [26]:
year = ['2000', '2010', '2015', '2020'] 
api_configs = [
    {
        "name": "Age_Data",
        "url": "https://www.onemap.gov.sg/api/public/popapi/getPopulationAgeGroup"
    }
]
for api_config in api_configs:
    for year in years:
        if exception_occurred:
            print("Script stopped due to an earlier exception.")
            break
        process_year(api_config, year)

Completed: BEDOK in 2000 has been logged.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Completed: CENTRAL WATER CATCHMENT in 2000 has been logged.
Completed: DOWNTOWN CORE in 2000 has been logged.
Completed: CHANGI in 2000 has been logged.
Completed: CHANGI BAY in 2000 has been logged.
Completed: LIM CHU KANG in 2000 has been logged.
Completed: BOON LAY in 2000 has been logged.
Completed: WESTERN WATER CATCHMENT in 2000 has been logged.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Completed: NEWTON in 2000 has been logged.
Completed: NORTH-EASTERN ISLANDS in 2000 has been logged.
Completed: ORCHARD in 2000 has been logged.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Completed: PUNGGOL in 2000 has been logged.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been log

## Asynchronous Multi-Threaded Requests (Religion Data)

In [27]:
year = ['2000', '2010', '2015', '2020'] 
api_configs = [
    {
        "name": "Religion_Data",
        "url": "https://www.onemap.gov.sg/api/public/popapi/getReligion"
    }
]
for api_config in api_configs:
    for year in years:
        if exception_occurred:
            print("Script stopped due to an earlier exception.")
            break
        process_year(api_config, year)

Completed: BEDOK in 2000 has been logged.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Skipped: CENTRAL WATER CATCHMENT in 2000 has no results.
Completed: DOWNTOWN CORE in 2000 has been logged.
Completed: CHANGI in 2000 has been logged.
Skipped: CHANGI BAY in 2000 has no results.
Skipped: LIM CHU KANG in 2000 has no results.
Skipped: BOON LAY in 2000 has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 has no results.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Completed: NEWTON in 2000 has been logged.
Skipped: NORTH-EASTERN ISLANDS in 2000 has no results.
Skipped: ORCHARD in 2000 has no results.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Skipped: PUNGGOL in 2000 has no results.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been logged.
Skipped: SIMPANG in

## Asynchronous Multi-Threaded Requests (Spoken Language Data)

In [28]:
year = ['2000', '2010', '2015', '2020'] 
api_configs = [
    {
        "name": "Spoken_Language_Data",
        "url": "https://www.onemap.gov.sg/api/public/popapi/getSpokenAtHome"
    }
]
for api_config in api_configs:
    for year in years:
        if exception_occurred:
            print("Script stopped due to an earlier exception.")
            break
        process_year(api_config, year)

Completed: BEDOK in 2000 has been logged.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Skipped: CENTRAL WATER CATCHMENT in 2000 has no results.
Completed: DOWNTOWN CORE in 2000 has been logged.
Completed: CHANGI in 2000 has been logged.
Skipped: CHANGI BAY in 2000 has no results.
Skipped: LIM CHU KANG in 2000 has no results.
Skipped: BOON LAY in 2000 has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 has no results.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Completed: NEWTON in 2000 has been logged.
Skipped: NORTH-EASTERN ISLANDS in 2000 has no results.
Skipped: ORCHARD in 2000 has no results.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Skipped: PUNGGOL in 2000 has no results.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been logged.
Skipped: SIMPANG in

## Asynchronous Multi-Threaded Requests (Tenancy Data)

In [29]:
year = ['2000', '2010', '2015', '2020'] 
api_configs = [
    {
        "name": "Tenancy_Data",
        "url": "https://www.onemap.gov.sg/api/public/popapi/getTenancy"
    }
]
for api_config in api_configs:
    for year in years:
        if exception_occurred:
            print("Script stopped due to an earlier exception.")
            break
        process_year(api_config, year)

Completed: BEDOK in 2000 has been logged.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Skipped: CENTRAL WATER CATCHMENT in 2000 has no results.
Completed: DOWNTOWN CORE in 2000 has been logged.
Completed: CHANGI in 2000 has been logged.
Skipped: CHANGI BAY in 2000 has no results.
Skipped: LIM CHU KANG in 2000 has no results.
Skipped: BOON LAY in 2000 has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 has no results.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Completed: NEWTON in 2000 has been logged.
Skipped: NORTH-EASTERN ISLANDS in 2000 has no results.
Skipped: ORCHARD in 2000 has no results.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Skipped: PUNGGOL in 2000 has no results.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been logged.
Skipped: SIMPANG in

## Asynchronous Multi-Threaded Requests (Dwelling Type Household Data)

In [30]:
year = ['2000', '2010', '2015', '2020'] 
api_configs = [
    {
        "name": "Dwelling_Type_Household_Data",
        "url": "https://www.onemap.gov.sg/api/public/popapi/getTypeOfDwellingHousehold"
    }
]
for api_config in api_configs:
    for year in years:
        if exception_occurred:
            print("Script stopped due to an earlier exception.")
            break
        process_year(api_config, year)

Completed: BEDOK in 2000 has been logged.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Skipped: CENTRAL WATER CATCHMENT in 2000 has no results.
Completed: DOWNTOWN CORE in 2000 has been logged.
Completed: CHANGI in 2000 has been logged.
Skipped: CHANGI BAY in 2000 has no results.
Skipped: LIM CHU KANG in 2000 has no results.
Skipped: BOON LAY in 2000 has no results.
Skipped: WESTERN WATER CATCHMENT in 2000 has no results.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Completed: NEWTON in 2000 has been logged.
Skipped: NORTH-EASTERN ISLANDS in 2000 has no results.
Skipped: ORCHARD in 2000 has no results.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Skipped: PUNGGOL in 2000 has no results.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been logged.
Skipped: SIMPANG in

## Asynchronous Multi-Threaded Requests (Dwelling Type Population Data)

In [31]:
year = ['2000', '2010', '2015', '2020'] 
api_configs = [
    {
        "name": "Dwelling_Type_Population_Data",
        "url": "https://www.onemap.gov.sg/api/public/popapi/getTypeOfDwellingPop"
    }
]
for api_config in api_configs:
    for year in years:
        if exception_occurred:
            print("Script stopped due to an earlier exception.")
            break
        process_year(api_config, year)

Completed: BEDOK in 2000 has been logged.
Completed: BUKIT TIMAH in 2000 has been logged.
Completed: BUKIT MERAH in 2000 has been logged.
Completed: BUKIT BATOK in 2000 has been logged.
Completed: CENTRAL WATER CATCHMENT in 2000 has been logged.
Completed: DOWNTOWN CORE in 2000 has been logged.
Completed: CHANGI in 2000 has been logged.
Completed: CHANGI BAY in 2000 has been logged.
Completed: LIM CHU KANG in 2000 has been logged.
Completed: BOON LAY in 2000 has been logged.
Completed: WESTERN WATER CATCHMENT in 2000 has been logged.
Completed: WOODLANDS in 2000 has been logged.
Completed: MARINE PARADE in 2000 has been logged.
Completed: NEWTON in 2000 has been logged.
Completed: NORTH-EASTERN ISLANDS in 2000 has been logged.
Completed: ORCHARD in 2000 has been logged.
Completed: PASIR RIS in 2000 has been logged.
Skipped: PIONEER in 2000 has no results.
Completed: PUNGGOL in 2000 has been logged.
Completed: QUEENSTOWN in 2000 has been logged.
Completed: SEMBAWANG in 2000 has been log

# The End.